In [8]:
from janome.tokenizer import Tokenizer
from gensim.models import word2vec
import re
import requests
from bs4 import BeautifulSoup

In [27]:
base_url = "https://www.uta-net.com"
target_url = 'https://www.uta-net.com/search/?Aselect=1&Bselect=4&Keyword=%E5%B1%B1%E5%8F%A3%E7%99%BE%E6%81%B5&sort=4'
music_num = 200

r = requests.get(target_url)

soup = BeautifulSoup(r.text, "html.parser")
url_list = []
#曲一覧から各曲のURLを取り出してリストに入れる
for i in range(music_num):
   href = soup.find_all("td", attrs={"class": "side td1"})[i].contents[0].get("href")
   url_list.append(href)         

kashi = ""
#曲ごとにRequestを送り歌詞を抽出する
for i in range(music_num):
   target_url = base_url + url_list[i]
   r = requests.get(target_url)
   soup = BeautifulSoup(r.text, "html.parser")

   for string in soup.find_all("div", attrs={"id": "kashi_area"})[0].strings:
       kashi += string

with open('./kashi_txt/momoe_kashi_200', mode = 'w', encoding = 'utf-8') as fw:
   fw.write(kashi)

In [28]:
import re
# 英数字の削除
kashi = re.sub("[a-xA-Z0-9_]","",kashi)
# 記号の削除
kashi = re.sub("[!-/:-@[-`{-~]","",kashi)
# 空白・改行の削除
kashi = re.sub(u'\n\n', '\n', kashi)
kashi = re.sub(u'\r', '', kashi)

In [29]:
def tokenize(text):
    t = Tokenizer()
    tokens = t.tokenize(text)
    word = []
    stop_word = create_stop_word()
    for token in tokens:
        part_of_speech = token.part_of_speech.split(",")[0]
        if part_of_speech == "名詞":
            if not token.surface in stop_word:
                word.append(token.surface)        
        if part_of_speech == "動詞":
            if not token.base_form in stop_word:
                word.append(token.base_form)
        if part_of_speech == "形容詞":
            if not token.base_form in stop_word:
                word.append(token.base_form)        
        if part_of_speech == "形容動詞":        
            if not token.base_form in stop_word:
                word.append(token.base_form)

    return word

In [30]:
def create_stop_word():
    target_url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    r =requests.get(target_url)
    soup=BeautifulSoup(r.text, "html.parser")
    stop_word=str(soup).split()
     #自分で追加
    my_stop_word=['れる','どの','いく','の','られる']
    stop_word.extend(my_stop_word)
    return stop_word

In [31]:
sentence = [tokenize(kashi)]
model = word2vec.Word2Vec(sentence, size=200, min_count=4, window=4, iter=50)
print(model.wv.most_similar(positive=[u"人生"], topn=10))

[('終り', 0.986318051815033), ('短い', 0.9854386448860168), ('愛し合う', 0.9821532964706421), ('生まれる', 0.9803616404533386), ('思い過ごす', 0.9801797270774841), ('ただ', 0.9794270992279053), ('暗い', 0.9787865877151489), ('閉じる', 0.9771473407745361), ('あける', 0.976866602897644), ('失う', 0.9767454266548157)]
